In [5]:
source("/Users/wyx/Desktop/dissertation/wholecode.R")

In [6]:
library(xgboost)
library(pROC)
library(DescTools)
library(ModelMetrics)
library(rms)

Warning message:
“package ‘xgboost’ was built under R version 4.3.3”

Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice



Attaching package: ‘ModelMetrics’


The following object is masked from ‘package:pROC’:

    auc


The following object is masked from ‘package:base’:

    kappa




In [7]:
generate_covariate_shifted_data_age_2 <- function(seed, age_mean_start, age_mean_end) {
  set.seed(seed)
  data <- generate_covariate_shifted_data_age(
    time_periods = 2,
    n = 9602,
    target_mortality_rate = 0.2,
    age_mean_start = age_mean_start,
    age_mean_end = age_mean_end,
    age_sd = 12.5,
    sex_prob = 0.309,
    cpd_prob = 0.107,
    pcs_prob = 0.282,
    cps_prob = 0.241, 
    beta_age = 0.0285181,
    beta_sex = 0.2196434,
    beta_cpd = 0.1886564,
    beta_pcs = 1.118599,
    beta_cps = 1.086517
  )
  return(data)
}

In [8]:
experiments_cs_age <- list(
  list(name = "age mean: 64.6000,64.6000", age_mean_start = 64.6, age_mean_end = 64.6),
  list(name = "age mean: 64.6000,63.8923", age_mean_start = 64.6, age_mean_end = 63.8923),
  list(name = "age mean: 64.6000,63.1923", age_mean_start = 64.6, age_mean_end = 63.1923),
  list(name = "age mean: 64.6000,62.5000", age_mean_start = 64.6, age_mean_end = 62.5),
  list(name = "age mean: 64.6000,61.8153", age_mean_start = 64.6, age_mean_end = 61.8153),
  list(name = "age mean: 64.6000,61.1381", age_mean_start = 64.6, age_mean_end = 61.1381),
  list(name = "age mean: 64.6000,60.4683", age_mean_start = 64.6, age_mean_end = 60.4683),
  list(name = "age mean: 64.6000,59.8058", age_mean_start = 64.6, age_mean_end = 59.8058),
  list(name = "age mean: 64.6000,59.1506", age_mean_start = 64.6, age_mean_end = 59.1506),
  list(name = "age mean: 64.6000,58.5026", age_mean_start = 64.6, age_mean_end = 58.5026)
)

In [9]:
# 循环200个随机种子，生成数据并计算性能指标
set.seed(123)  # 确保结果可重复
num_simulations <- 200

results_event_rate_shift_ml <- data.frame()

for (exp in experiments_cs_age) {
    results <- data.frame(auc = numeric(num_simulations), o_e = numeric(num_simulations), c_slope = numeric(num_simulations), brier_score = numeric(num_simulations))
    for (i in 1:num_simulations) {
      # 设置随机种子
      set.seed(i)

      data_shifted_xgb <- generate_covariate_shifted_data_age_2(i, exp$age_mean_start, exp$age_mean_end)

      # 确保 mortality 变量是因子类型
      data_shifted_xgb$mortality <- as.factor(data_shifted_xgb$mortality)
      data_val$mortality <- as.factor(data_val$mortality)

      # 转换数据格式
      dtrain <- xgb.DMatrix(data = as.matrix(data_shifted_xgb[, -6]), label = as.numeric(data_shifted_xgb$mortality) - 1)
      dval <- xgb.DMatrix(data = as.matrix(data_val[, -6]), label = as.numeric(data_val$mortality) - 1)

      # 设置参数
      params <- list(
        booster = "gbtree",
        objective = "binary:logistic",
        eta = 0.1,
        max_depth = 3,
        eval_metric = "auc"
      )

      # 训练模型
      xgb_model <- xgb.train(params, dtrain, nrounds = 50)


      # 预测
      predictions <- predict(xgb_model, dval)

      # 确保测试集的mortality变量是二进制数值类型
      test_mortality <- as.numeric(data_val$mortality) - 1

      # 计算 AUC
      auc <- roc(data_val$mortality, predictions)$auc
      print(paste("auc:", auc))

      # 绘制 ROC 曲线
      #plot.roc(roc_obj, main = paste("ROC Curve (AUC =", round(auc_value, 2), ")"), col = "blue")

       # 计算O/E
      o_e <- sum(test_mortality) / sum(predictions)
      print(paste("O/E 比率:", o_e))

      # 计算校准斜率 (C-slope)
      # 绘制校准曲线并提取斜率
      calibration <- val.prob(predictions, test_mortality, xlab="Predicted Probability", ylab="Observed Probability", pl = FALSE, smooth = FALSE)

      c_slope <- calibration["Slope"]

      print(paste("c_slope:", c_slope))

      # 计算 Brier 分数
      brier_score <- BrierScore(test_mortality, predictions)
      print(paste("Brier 分数:", brier_score))

      # 保存结果
      results[i, ] <- c(auc = auc, o_e = o_e, c_slope = c_slope, brier_score = brier_score)

    }

    # 计算中位数、25%和75%的性能指标
    median_metrics_event_rate_shift_ml <- apply(results, 2, median)
    quantile_25_metrics_event_rate_shift_ml <- apply(results, 2, quantile, probs = 0.25)
    quantile_75_metrics_event_rate_shift_ml <- apply(results, 2, quantile, probs = 0.75)

    # 将实验名称和结果添加到总的结果数据框
    summary_metrics_event_rate_shift_ml <- data.frame(
    experiment = exp$name,
    auc_median = median_metrics_event_rate_shift_ml['auc'],
    auc_25 = quantile_25_metrics_event_rate_shift_ml['auc'],
    auc_75 = quantile_75_metrics_event_rate_shift_ml['auc'],
    o_e_median = median_metrics_event_rate_shift_ml['o_e'],
    o_e_25 = quantile_25_metrics_event_rate_shift_ml['o_e'],
    o_e_75 = quantile_75_metrics_event_rate_shift_ml['o_e'],
    c_slope_median = median_metrics_event_rate_shift_ml['c_slope'],
    c_slope_25 = quantile_25_metrics_event_rate_shift_ml['c_slope'],
    c_slope_75 = quantile_75_metrics_event_rate_shift_ml['c_slope'],
    brier_score_median = median_metrics_event_rate_shift_ml['brier_score'],
    brier_score_25 = quantile_25_metrics_event_rate_shift_ml['brier_score'],
    brier_score_75 = quantile_75_metrics_event_rate_shift_ml['brier_score']
    )
    
    results_event_rate_shift_ml <- rbind(results_event_rate_shift_ml, summary_metrics_event_rate_shift_ml)
    
}




Setting levels: control = 0, case = 1

Setting direction: controls < cases



[1] "auc: 0.721773029066186"
[1] "O/E 比率: 1.00492761969956"
[1] "c_slope: 1.08214770411894"
[1] "Brier 分数: 0.144193146413868"


Setting levels: control = 0, case = 1

Setting direction: controls < cases



[1] "auc: 0.723537064337265"
[1] "O/E 比率: 0.955001966760982"
[1] "c_slope: 1.19941220580017"
[1] "Brier 分数: 0.143980096355089"


Setting levels: control = 0, case = 1

Setting direction: controls < cases



...

[1] "auc: 0.722114765776272"
[1] "O/E 比率: 0.925736062744174"
[1] "c_slope: 1.11917892117512"
[1] "Brier 分数: 0.143974709323186"


Setting levels: control = 0, case = 1

Setting direction: controls < cases



[1] "auc: 0.72318913923605"
[1] "O/E 比率: 0.962394276097503"
[1] "c_slope: 1.13946582880674"
[1] "Brier 分数: 0.14404756526715"


Setting levels: control = 0, case = 1

Setting direction: controls < cases



[1] "auc: 0.721042455113536"
[1] "O/E 比率: 0.951039763648241"
[1] "c_slope: 1.12217149679022"
[1] "Brier 分数: 0.144351378003685"


In [10]:
results_event_rate_shift_ml

,experiment,auc_median,auc_25,auc_75,o_e_median,o_e_25,o_e_75,c_slope_median,c_slope_25,c_slope_75,brier_score_median,brier_score_25,brier_score_75
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
auc,"age mean: 64.6000,64.6000",0.7216787,0.7203728,0.7232080,0.9908482,0.9761388,1.0064552,1.101810,1.082535,1.129009,0.1439825,0.1437168,0.1442527
auc1,"age mean: 64.6000,63.8923",0.7217880,0.7204313,0.7233237,0.9844732,0.9690526,1.0012505,1.097802,1.076876,1.130247,0.1439675,0.1437108,0.1442354
auc2,"age mean: 64.6000,63.1923",0.7219143,0.7206225,0.7232823,0.9814085,0.9659025,0.9950010,1.100211,1.077746,1.133578,0.1439832,0.1437321,0.1442794
auc3,"age mean: 64.6000,62.5000",0.7220551,0.7207519,0.7234771,0.9750750,0.9610075,0.9909535,1.098982,1.079372,1.125148,0.1439983,0.1436773,0.1442241
auc4,"age mean: 64.6000,61.8153",0.7219815,0.7206711,0.7235735,0.9698941,0.9551928,0.9843047,1.099926,1.080812,1.127355,0.1440235,0.1437316,0.1442795
auc5,"age mean: 64.6000,61.1381",0.7222837,0.7205115,0.7237230,0.9651909,0.9516392,0.9790055,1.104716,1.077892,1.133725,0.1440019,0.1436860,0.1442618
auc6,"age mean: 64.6000,60.4683",0.7221471,0.7206978,0.7235719,0.9588548,0.9452016,0.9752148,1.107238,1.082709,1.131664,0.1440001,0.1436941,0.1442567
auc7,"age mean: 64.6000,59.8058",0.7223812,0.7211003,0.7238721,0.9535999,0.9398469,0.9694069,1.111393,1.083655,1.134098,0.1440016,0.1436587,0.1442606
auc8,"age mean: 64.6000,59.1506",0.7223537,0.7210897,0.7237548,0.9494069,0.9360897,0.9644379,1.107503,1.084818,1.133216,0.1440594,0.1436885,0.1443369
